## Project setup

In [1]:
import numpy as np
import pandas as pd
import logging
from prophet import Prophet
from prophet.make_holidays import make_holidays_df


Importing plotly failed. Interactive plots will not work.


In [2]:
# CONFIGURATION & CONSTANTS
receivals = pd.read_csv('../data/kernel/receivals.csv')
purchase_orders = pd.read_csv("../data/kernel/purchase_orders.csv")
mapping = pd.read_csv("../data/prediction_mapping.csv")

logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

pred_days = 150 # number of days to predict
start_date="2009-01-01" # start date for continuos training data 
min_factor = 0.6 #minimum shrink factor (applied to the lowest annual quantity items)
max_factor = 0.95 #maximum shrink factor (applied to the highest annual quantity items)
changepoint_prior_scale= 0.008 #changepoint prior scale for prophet model

## Preparation functions

In [3]:
def receival_cleaned(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.dropna(subset=["rm_id","date_arrival", "net_weight"]).copy()
    df['date_arrival'] = pd.to_datetime(df['date_arrival'], utc=True)
    cols = ["rm_id", "product_id", "date_arrival", "net_weight"]

    return df[cols]


def po_cleaned(df: pd.DataFrame) -> pd.DataFrame:
    
    df["delivery_date"] = pd.to_datetime(df["delivery_date"], errors="coerce", utc=True)
    df["delivery_date"] = pd.to_datetime(df["delivery_date"]).dt.date
    df = df.dropna(subset=["purchase_order_id", "delivery_date", "quantity"])
    df = df[df["quantity"] > 0]
    cols = ["delivery_date", "product_id", "quantity"]

    return df[cols]

def quantity_agg(df: pd.DataFrame) -> pd.DataFrame:

    df_agg = (
        df.groupby(["delivery_date", "product_id"])["quantity"]
        .sum()
        .reset_index()
        .rename(columns={"delivery_date": "date", "quantity": "quantity_ordered"})
    )

    return df_agg

def daily_weight_agg(df: pd.DataFrame) -> pd.DataFrame:

    df["date"] = df["date_arrival"].dt.floor("D")
    df_agg = (
        df.groupby(["rm_id", "date"], as_index=False)["net_weight"]
        .sum()
        .rename(columns={"net_weight": "daily_weight"})
    )
    prod_map = df.groupby("rm_id")["product_id"].first()
    df_agg["product_id"] = df_agg["rm_id"].map(prod_map)

    return df_agg

def orders_timeseries_merged(timeseries_df: pd.DataFrame, orders_agg_df: pd.DataFrame) -> pd.DataFrame:

    final_df = pd.merge(timeseries_df, orders_agg_df, on=["date", "product_id"], how="left")
    final_df["quantity_ordered"] = final_df["quantity_ordered"].fillna(0)
    final_df["date"] = pd.to_datetime(final_df["date"])
    cols = ["rm_id", "product_id", "date", "daily_weight", "quantity_ordered"]

    return final_df[cols]

def timeseries_continuous(df: pd.DataFrame, start_date, end_date) -> pd.DataFrame:
   
    prod_map = df.groupby("rm_id")["product_id"].first()
    all_rms = df["rm_id"].unique()
    all_dates = pd.date_range(start=start_date, end=end_date, freq="D", tz="UTC")
    
    full_index = pd.MultiIndex.from_product([all_rms, all_dates], names=["rm_id", "date"])
    df = df.set_index(["rm_id", "date"]).reindex(full_index).reset_index()
  
    df["product_id"] = df["rm_id"].map(prod_map)
    df["daily_weight"] = df["daily_weight"].fillna(0)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    
    return df

def is_holiday(date, holiday_df):
    return date in set(holiday_df['ds'])


## Preprocessing

In [4]:
receivals = receival_cleaned(receivals)
receivals_daily_agg = daily_weight_agg(receivals)

timeseries = timeseries_continuous(receivals_daily_agg, start_date=start_date, end_date="2025-01-01")

receivals["date_arrival"] = pd.to_datetime(receivals["date_arrival"])
receivals_after = receivals[receivals["date_arrival"] >= "2024-10-01"]
active_ids = set(receivals_after["rm_id"].unique())
rm_counts = receivals['rm_id'].value_counts()
ids_enough = rm_counts[rm_counts >= 10].index

purchase_orders = po_cleaned(purchase_orders)
purchase_orders['delivery_date'] = pd.to_datetime(purchase_orders['delivery_date'], utc=True)
purchase_orders['year'] = purchase_orders['delivery_date'].dt.year

quantity_year = (
    purchase_orders
    .groupby(['product_id', 'year'], as_index=False)['quantity']
    .sum()
    .rename(columns={'quantity': 'quantity_year'})
)

df = timeseries.copy()
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year

df = df.merge(quantity_year, on=['product_id', 'year'], how='left')

df["cum_weight"] = df.groupby(['rm_id'])['daily_weight'].cumsum()

df = df [["rm_id", "date", "quantity_year", "cum_weight"]]
df['quantity_year']=df['quantity_year'].fillna(0)

df = df[df['rm_id'].isin(active_ids)]
df = df[df['rm_id'].isin(ids_enough)]

## Prophet training

In [5]:
results = []
for rm_id, group in df.groupby('rm_id'):

    tmp = group.rename(columns={"date": "ds", "cum_weight": "y"})
    active_first = tmp.loc[tmp['y'] > 0, 'ds'].iloc[0]
    norway_holidays = make_holidays_df(year_list=tmp['ds'].dt.year.unique(), country='NO')
    tmp = tmp[tmp['ds'] >= active_first].reset_index(drop=True)
    tmp['working_days'] = (tmp['ds'].dt.weekday < 5).astype(int)

    
    m = Prophet(
        weekly_seasonality = True,
        holidays = norway_holidays,
        
        changepoint_prior_scale = changepoint_prior_scale,  
        seasonality_prior_scale = 10.0,      
        seasonality_mode = 'additive' 
        )
    
    m.add_regressor('working_days')
   
    m.fit(tmp)

    future_df = m.make_future_dataframe(periods=150, freq="D")
    future_df['working_days'] = future_df['ds'].dt.weekday < 5
    future_df['working_days'] = future_df['working_days'].astype(int)
    future_df['quantity_year'] = tmp.iloc[-1]['quantity_year']
        
    forecast = m.predict(future_df)

    forecast['rm_id'] = rm_id
    forecast['quantity_year'] = tmp.iloc[-1]['quantity_year']
    results.append(forecast[['ds', 'rm_id', 'quantity_year', 'yhat','yhat_lower']])
final = pd.concat(results)

final = final.merge(df[['rm_id', 'date', 'cum_weight']], left_on=['rm_id', 'ds'], right_on=['rm_id', 'date'], how='left')
final = final.drop(columns=['date'])
final = final.rename(columns={'cum_weight': 'y'})


13:56:12 - cmdstanpy - INFO - Chain [1] start processing
13:56:13 - cmdstanpy - INFO - Chain [1] done processing
13:56:14 - cmdstanpy - INFO - Chain [1] start processing
13:56:17 - cmdstanpy - INFO - Chain [1] done processing
13:56:18 - cmdstanpy - INFO - Chain [1] start processing
13:56:20 - cmdstanpy - INFO - Chain [1] done processing
13:56:21 - cmdstanpy - INFO - Chain [1] start processing
13:56:22 - cmdstanpy - INFO - Chain [1] done processing
13:56:23 - cmdstanpy - INFO - Chain [1] start processing
13:56:25 - cmdstanpy - INFO - Chain [1] done processing
13:56:26 - cmdstanpy - INFO - Chain [1] start processing
13:56:29 - cmdstanpy - INFO - Chain [1] done processing
13:56:30 - cmdstanpy - INFO - Chain [1] start processing
13:56:32 - cmdstanpy - INFO - Chain [1] done processing
13:56:33 - cmdstanpy - INFO - Chain [1] start processing
13:56:35 - cmdstanpy - INFO - Chain [1] done processing
13:56:36 - cmdstanpy - INFO - Chain [1] start processing
13:56:38 - cmdstanpy - INFO - Chain [1]

## Post Processing

In [6]:
future_df_2025 = final[(final['ds'] >= '2025-01-02') & (final['ds'] <= '2025-12-31')].copy()

preds = future_df_2025["yhat"].values.copy()

for start in range(0, len(preds), pred_days):
    end = start + pred_days
    preds[start:end] = np.maximum.accumulate(preds[start:end])
    
future_df_2025["yhat_corr"] = preds

preds = future_df_2025['yhat_corr'].values.copy()
predicted_weight = np.zeros_like(preds)

for start in range(0, len(preds), pred_days):
    end = start + pred_days
    base = preds[start]                   
    predicted_weight[start:end] = preds[start:end] - base

future_df_2025['predicted_weight'] = predicted_weight


In [7]:
future_df_2025 = future_df_2025[(future_df_2025['ds'] >= '2025-01-02')]

q_min = future_df_2025['quantity_year'].min()
q_max = future_df_2025['quantity_year'].max()
future_df_2025['quantity_year_norm'] = (future_df_2025['quantity_year'] - q_min) / (q_max - q_min)

future_df_2025['shrinking'] = min_factor + (max_factor - min_factor) * future_df_2025['quantity_year_norm']


future_df_2025['predicted_weight'] = (
    future_df_2025['predicted_weight'] * future_df_2025['shrinking']
)

future_df_2025 = future_df_2025[['ds','predicted_weight','rm_id']]
future_df_2025 = future_df_2025.rename(columns={'ds': 'date'})


## Submission

In [ ]:
mapping["forecast_start_date"] = pd.to_datetime(mapping["forecast_start_date"], utc=True)
mapping["forecast_end_date"] = pd.to_datetime(mapping["forecast_end_date"], utc=True)
mapping = mapping.rename(columns={'forecast_end_date': 'date'})
future_df_2025["date"] = pd.to_datetime(future_df_2025["date"], utc=True)

mapping = mapping.merge(
    future_df_2025[['rm_id', 'date', 'predicted_weight']],
    on=['rm_id', 'date'],
    how='left'
)

submission = mapping[["ID", "predicted_weight"]].copy()

submission["predicted_weight"] = submission["predicted_weight"].fillna(0)

submission.to_csv("../submissions/final_submission_prophet.csv", index=False)
print("Finished. Submission file saved in model.")



Finished. Submission file saved in model.
